In [ ]:
try:
    from isaacgym import gymtorch
    from isaacgym import gymapi
except ImportError:
    pass
from bc_algos.dataset.robomimic import RobomimicDataset
from bc_algos.envs.robosuite import RobosuiteEnv
from bc_algos.dataset.isaac_gym import IsaacGymDataset
from bc_algos.envs.isaac_gym import IsaacGymEnv
from bc_algos.utils.constants import Modality
import bc_algos.utils.constants as Constants
import omegaconf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

# change this to test different simulator environments
type = Constants.EnvType.ISAAC_GYM

### Dataset Init

In [ ]:
obs_key_to_modality = {
    "robot0_eef_pos": Modality.LOW_DIM,
    "robot0_eef_quat": Modality.LOW_DIM,
    "agentview_image": Modality.RGB
}
obs_group_to_key = {
    "obs": ["robot0_eef_pos", "robot0_eef_quat", "agentview_image"],
    "goal": ["agentview_image"]
}
action_key = "actions"

if type == Constants.EnvType.ROBOSUITE:
    path = "../datasets/test/square_ph.hdf5"
    demo_id = "demo_0"

elif type == Constants.DatasetType.ISAAC_GYM:
    path = "../datasets/0602_1000_full"
    demo_id = 0

In [ ]:
if type == Constants.EnvType.ROBOSUITE:
    dataset = RobomimicDataset(
        path=path,
        obs_key_to_modality=obs_key_to_modality,
        obs_group_to_key=obs_group_to_key,
        action_key=action_key,
        history=0,
        action_chunk=1,
        pad_history=False,
        pad_action_chunk=False,
        get_pad_mask=False,
        demo_ids=[demo_id],
    )
if type == Constants.EnvType.ISAAC_GYM:
    dataset = IsaacGymDataset(
        path=path,
        obs_key_to_modality=obs_key_to_modality,
        obs_group_to_key=obs_group_to_key,
        action_key=action_key,
        history=0,
        action_chunk=1,
        pad_history=False,
        pad_action_chunk=False,
        get_pad_mask=False,
        demo_ids=[demo_id],
    )

### Environment Init

In [ ]:
if type == Constants.EnvType.ROBOSUITE:
    env_meta = json.loads(dataset.hdf5_file["data"].attrs["env_args"])
    env = RobosuiteEnv(
        env_name=env_meta["env_name"],
        obs_key_to_modality=obs_key_to_modality,
        render=False,
        use_image_obs=True,
        use_depth_obs=False,
        **env_meta["env_kwargs"],
    )
elif type == Constants.EnvType.ISAAC_GYM:
    env_cfg_path = "../config/isaac_gym_env.json"
    config = omegaconf.OmegaConf.create(json.load(open(env_cfg_path, "r")))
    env = IsaacGymEnv(
        env_name=config.task.name,
        obs_key_to_modality=obs_key_to_modality,
        render=False,
        use_image_obs=True,
        use_depth_obs=False,
        config=config,
    )

### Test Environment

In [ ]:
# initialize environment to same simulation state at the beginning of the demo
if type == Constants.EnvType.ROBOSUITE:
    xml = dataset.hdf5_file[f"data/{demo_id}"].attrs["model_file"]
    env.load_env(xml=xml)
    init_state = dataset.hdf5_file[f"data/{demo_id}/states"][0]
    env.reset_to(state=init_state)
elif type == Constants.EnvType.ISAAC_GYM:
    run = pd.read_pickle(dataset.demo_id_to_run_path(demo_id=demo_id))
    metadata = run["metadata"]
    cubes_pos = run["obs"]["cubes_pos"][0]
    cubes_quat = run["obs"]["cubes_quat"][0]
    cubes_pose = np.concatenate([cubes_pos, cubes_quat], axis=-1)
    metadata.update({
        "block_init_pose": cubes_pose,
        "start_q": run["obs"]["q"][0],
    })
    env.reset_to(state=metadata)

In [ ]:
# compare reference and executed trajectories visually
demo_length = dataset.demo_len(demo_id=demo_id)
demo = dataset.load_demo(demo_id=demo_id)
for i in range(demo_length):
    frame = dataset.seq_from_timstep(demo_id=demo_id, demo=demo, t=i)
    action = frame[action_key][0]
    ref_image = np.transpose(frame["obs"]["agentview_image"][0], (1, 2, 0))
    img = env.render()
    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(ref_image)
    axs[0].set_title("Reference")
    axs[1].imshow(img)
    axs[1].set_title("Execution")
    plt.savefig(f"../outputs/test_env/{i}.jpg")
    plt.draw()
    plt.close()
    env.step(action=action)